## 실습문제 1 - 1

이전에 진행중인 movies 를 그대로 이어서 진행한다.  

아래 사진과 같이 메인 페이지에 전체 영화 데이터를 불러와서 출력한다.  
(데이터의 개수에 따라 목록의 개수가 변해야하며, 관리자 계정으로 데이터를 추가/삭제하면서 테스트한다.)

<img src="practice_images/image19.jpg" width="30%" height="1%"/>

1. movies/views.py 의 index 함수 수정 (리스트를 템플릿에 전달)

    ```python
    from .models import Movie

    def index(request):
        # movie = request.GET.get('movie_pk')
        movie_list = Movie.objects.all()
        context = {
            # 'movie': movie_pk
            'movie_list': movie_list
        }
        return render(request, 'movies/index.html', context)
    ```

2. movies/index.html 파일 수정 (주어진 이미지에 맞게 수정)
    ```html
    {% extends "base.html" %}
    {% block content %}
    <h1>영화 블로그 프로젝트 메인 페이지</h1>
    <p>영화 블로그 내용을 관리합니다.</p>
    <ul>
    {% for movie in movie_list %}
    <li>
        <a href="{% url "movies:movie_detail" movie.pk %}">{{ movie.title }}</a>
        <p>개봉여부:       
        {% if movie.is_open %}
            개봉 완료
        {% else %}
            개봉 전
        {% endif %}</p> 
    </li>
    {% empty %}
    <li>영화가 없습니다.</li>
    {% endfor %}

    </ul>
    <a href="/movies/create_movie/">영화 추가</a>
    {% endblock content %}
    ```

## 실습문제 1 - 2

이전에 진행중인 movies 를 그대로 이어서 진행한다.  

영화 목록에서 제목을 클릭할 경우, detail 페이지로 이동하도록 한다. 

영화 detail 페이지는 아래 이미지와 같도록 구성한다. (url 도 아래 이미지와 같도록 구성한다.)

<img src="practice_images/image20.jpg" width="30%" height="1%"/>

1. movies/views.py 의 movie_detail 함수 수정 
    ```python
    def movie_detail(request, movie):
        movie = Movie.objects.get(pk=movie)
        context = {
            'movie': movie
        }
        return render(request, 'movies/movie_detail.html', context)
    ```

2. movies/movie_detail.html 수정 ( 주어진 상세 페이지에 맞게)
    ```html 
    {% extends "base.html" %}

    {% block content %}
    <h1>영화 상세 페이지</h1>
    <hr>
    <p>{{ movie.id }}번 째 영화</p>
    <p>영화 제목 : {{ movie.title }}</p>
    <p>영화 콘텐츠 : {{ movie.content }}</p>
    <p>영화 제작일 : {{ movie.created_at}} </p>
    <p>영화 개봉 여부 : {{ movie.is_open }}</p>
    <hr>
    {% endblock content %}
    ```

## 실습문제 1 - 3

이전에 진행중인 movies 를 그대로 이어서 진행한다.  

영화 추가 페이지를 아래 이미지와 같이 구성하고, 제출을 누르면 영화 데이터가 생성되고, 생성된 영화의 상세페이지로 이동하도록 구성한다. 

<img src="practice_images/image21.jpg" width="30%" height="1%"/>
<img src="practice_images/image23.jpg" width="30%" height="1%"/>
<img src="practice_images/image24.jpg" width="30%" height="1%"/>

1. movies/create_movie.html 수정 (영화 생성 페이지 수정)
    ```html
    {% extends "base.html" %}
    {% block content %}
    <title>영화 생성 페이지</title>
    <h1>이 곳에서 영화를 생성합니다.</h1>

    <form action="{% url "movies:save_movie" %}" method="POST">
    {% csrf_token %}
    <label for="movie_id">영화제목 : </label>
    <input type="text" name="movie_title" id="movie_id"> <br>
    <label for="movie_content">영화 콘텐츠 : </label>
    <textarea name="movie_content" id="movie_content" cols="30" rows="10"></textarea>
    <input type="submit" value="제출">
    </form>
    <a href="/movies/">메인 페이지</a>
    {% endblock content %}
    ```

2. create_movie.html 에서 필요한 "movies:save_movie" url 생성  
movies/urls.py 의 urlpatterns에 아래 코드 추가
    ```python
    path('save_movie/', views.save_movie, name='save_movie'),
    ```
3. movies.urls.py 의 save_movie url에서 필요한 save_movie 함수 생성  
movies/views.py 에 save_movie 함수 생성 
    ```python
    def save_movie(request):
        movie_title = request.POST.get('movie_title')
        movie_content = request.POST.get('movie_content')
        is_open = False

        movie = Movie(title=movie_title, content=movie_content, is_open=is_open)
        movie.save()

        return redirect('movies:movie_detail', movie.pk)
    ```

## 실습문제 1 - 4

이전에 진행중인 movies 를 그대로 이어서 진행한다.  

영화 상세 페이지 가장 아래에 영화 데이터 삭제 input 을 생성하고, 삭제를 누르면 영화 데이터가 삭제되고 영화목록 페이지로 이동한다. 

<img src="practice_images/image25.jpg" width="30%" height="1%"/>
<img src="practice_images/image26.jpg" width="30%" height="1%"/>

1. movies/movie_detail.html 에서 영화 삭제 input 추가 
    ```html
    {% extends "base.html" %}

    {% block content %}
    <h1>영화 상세 페이지</h1>
    <hr>
    <p>{{ movie.id }}번 째 영화</p>
    <p>영화 제목 : {{ movie.title }}</p>
    <p>영화 콘텐츠 : {{ movie.content }}</p>
    <p>영화 제작일 : {{ movie.created_at}} </p>
    <p>영화 개봉 여부 : {{ movie.is_open }}</p>
    <hr>
    <form action="{% url "movies:delete_movie" movie.pk %}" method="POST">
        {% csrf_token %}
        <input type="submit" value="삭제">
    </form>  
    {% endblock content %}
    ```

2. movies/movie_detail.html 에서 필요한 "movies:delete_movie" url 생성  
movies/urls.py 의 urlpatterns에 아래 코드 추가 
    ```ptyhon
    path('delete_movie/<int:movie_pk>/', views.delete_movie, name='delete_movie'),
    ```

3. movies/urls.py의 delete_movie url이 필요한 delete_movie 함수 생성  
movies/views.py에 delete_movie 함수 생성
    ```python
    def delete_movie(request, movie_pk):
        movie = Movie.objects.get(pk=movie_pk)
        movie.delete()
        return redirect('movies:index')
    ```

## 실습문제 1 - 5

이전에 진행중인 movies 를 그대로 이어서 진행한다.  

영화 디테일 페이지에 영화 수정 태그를 추가하고, 해당 태그를 클릭할 경우 영화 수정 페이지로 이동한다.

영화를 수정할 경우, 수정한 내용에 따라 데이터가 수정되며 수정을 완료한 후에는 
수정한 영화의 상세 페이지로 이동한다. 
영화 수정 페이지는 아래 이미지와 같이 구성한다. 

<img src="practice_images/image27.jpg" width="30%" height="1%"/>
<img src="practice_images/image28.jpg" width="30%" height="1%"/>
<img src="practice_images/image29.jpg" width="30%" height="1%"/>

1. 영화 상세 페이지에 영화 수정 input 추가  
movies/movie_detail.html 수정하기
    ```html
    {% extends "base.html" %}

    {% block content %}
    <h1>영화 상세 페이지</h1>
    <hr>
    <p>{{ movie.id }}번 째 영화</p>
    <p>영화 제목 : {{ movie.title }}</p>
    <p>영화 콘텐츠 : {{ movie.content }}</p>
    <p>영화 제작일 : {{ movie.created_at}} </p>
    <p>영화 개봉 여부 : {{ movie.is_open }}</p>
    <hr>
    <form action="{% url "movies:update_movie" movie.pk %}" method="POST">
        {% csrf_token %}
        <input type="submit" value="수정">
    </form>
    <hr>
    <form action="{% url "movies:delete_movie" movie.pk %}" method="POST">
        {% csrf_token %}
        <input type="submit" value="삭제">
    </form>  
    {% endblock content %}
    ```

2. 영화 상세 페이지에서 필요한 "movies:update_movie" url을 생성  
movies/urls.py 의 urlpatterns에 아래 코드 추가
    ```python
    path('update_movie/<int:movie_pk>/', views.update_movie, name='update_movie')
    ```

3. movies/urls.py의 update_movie url이 필요한 update_movie 함수 생성  
movies/views.py에 update_movie 함수 작성 
    ```python
    def update_movie(request, movie_pk):
        movie = Movie.objects.get(pk=movie_pk)
        context = {
            'movie': movie
        }
        return render(request, 'movies/update_movie.html', context)
    ```

4. movies/views.py의 update_movie 함수가 랜더링하는 영화 수정 페이지 작성 (movies/update_movie.html 작성하기)
    ```html
    {% extends "base.html" %}

    {% block content %}
    <h1>이 곳에서 영화 정보를 수정합니다.</h1>

    <form action="{% url "movies:edit_movie" movie.pk %}" method="POST">
        {% csrf_token %}
        <label for="movie_title">work : </label>
        <input type="text" name="movie_title" id="movie_title" value="{{ movie.title }}"> <br>
        <label for="movie_content">content : </label>
        <textarea name="movie_content" id="movie_content" cols="30" rows="10">{{ movie.content }}</textarea>
        <input type="submit" value="수정">
    </form>
    {% endblock content %}
    ```

5. movies/update_movie.html에서 필요한 "movies:edit_movie" url 추가하기  
movies/urls.py 의 urlpatterns에 아래 코드 추가하기
    ```python
        path('edit_movie/<int:movie_pk>/', views.edit_movie, name='edit_movie')
    ```

6. movies/urls.py의 edit_movie url이 필요한 edit_movie 함수 작성하기  
movies/views.py 에 edit_movie 함수 작성하기 
    ```python
    def edit_movie(request, movie_pk):
        movie = Movie.objects.get(pk=movie_pk)

        movie_title = request.POST.get('movie_title')
        movie_content = request.POST.get('movie_content')

        movie.title = movie_title
        movie.content = movie_content
        movie.save()

        return redirect('movies:movie_detail', movie.pk)
    ```

## 실습문제 1 - 6

이전에 진행중인 movies 를 그대로 이어서 진행한다.  

영화 목록 페이지에서 텍스트로 표현되던 개봉 여부를 체크박스로 변경한다. (mdn 공식문서 보기)

개봉이 되었다면 체크박스에 체크가 나타나도록 한다. 

<img src="practice_images/image30.jpg" width="30%" height="1%"/>

1. movies/index.html 페이지 수정하기(이미지가 제시한 조건대로)
    ```html
    {% extends "base.html" %}
    {% block content %}
    <h1>영화 블로그 프로젝트 메인 페이지</h1>
    <p>영화 블로그 내용을 관리합니다.</p>
    <ul>
    {% for movie in movie_list %}
    <li>
        <a href="{% url "movies:movie_detail" movie.pk %}">{{ movie.title }}</a>
        <p>개봉여부:       
        <input type="checkbox" value="{{ movie.is_open }}" {% if movie.is_open %}checked{% endif %}>
    </li>
    {% empty %}
    <li>영화가 없습니다.</li>
    {% endfor %}
    </ul>
    <a href="/movies/create_movie/">영화 추가</a>
    {% endblock content %}
    ```
